# Info
: 직접적으로 '상승'과 '하락'과 유사도가 가장 높은 기사 키워드를 찾기위한 코드 방법은 아래와 같다.
1. 두 단어와 각각 유사도가 높은 단어들을 구한 후 양쪽에 모두 속하는 단어들을 제거한다.
2. 각각 유사한 단어들을 5개씩 구힌디.

In [1]:
import pandas as pd
import glob

file_name = glob.glob('../article/headline_noun_keyword*')

keyword_df = []
for file in file_name:
    df = pd.read_pickle(file)
    keyword_df.append(df)

keyword_all = pd.concat(keyword_df, ignore_index = True)
all_article = keyword_all[keyword_all['week'] <= 807]
all_article

,date,co,headline,week,pos
0,2015.08.09. 오후 11:59,머니투데이,미연준 금리 인상 예고 은행별 주택아파트담보대출 사이트로 해법 찾기,501,"[미연, 금리, 인상, 예고, 은행, 주택, 아파트, 담보, 대출, 사이트, 해법]"
1,2015.08.09. 오후 9:52,경향신문,지방 주택시장 과열 양상… 대구 1년 새 집값 11% 올라,501,"[지방, 주택, 시장, 과열, 양상, 대구, 년, 집값]"
2,2015.08.09. 오후 8:51,세계일보,전세 시황 계절 요인에… 치솟던 전세가 상승폭 주춤,501,"[전세, 시황, 계절, 요인, 전세, 상승, 폭]"
3,2015.08.09. 오후 8:51,세계일보,매매 시황 가계부채 대책 후폭풍… 매매가 상승 둔화,501,"[매매, 시황, 가계, 부채, 대책, 후폭풍, 매매, 상승, 둔화]"
4,2015.08.09. 오후 8:49,세계일보,너무 뜨거운 부동산 시장… 손 델라,501,"[부동산, 시장, 손]"
...,...,...,...,...,...
1067224,2015.08.08. 오전 12:02,서울경제,배후수요 풍부 향후 미래가치가 기대되는 천안 레이크타운2차 푸르지오 분양,500,"[배후, 수요, 풍부, 향후, 미래, 가치, 기대, 천안, 레이크, 타운, 차, 푸..."
1067225,2015.08.08. 오전 12:02,헤럴드경제,일산아이파크 수도권미분양아파트 모래알 진주찾기,500,"[일산, 아이파크, 수도, 미분양, 아파트, 모래알, 진주]"
1067226,2015.08.08. 오전 12:02,헤럴드경제,봉담 코오롱 하늘채 편리한 교통 및 우수한 교육환경 자랑,500,"[봉담, 코오롱, 하늘, 편리, 교통, 우수, 교육, 환경, 자랑]"
1067227,2015.08.08. 오전 12:00,경향신문,대명리조트 콘도 회원권으로 여름 휴가 알차게 보내는 방법,500,"[대명, 리조트, 콘, 회원, 여름, 휴가, 방법]"


In [2]:
tokenized_data = []
for keyword in all_article['pos']:
    tokenized_data.append(keyword)

In [3]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, window = 5, min_count = 500, workers = 4, sg = 1)

In [4]:
model.wv.vectors.shape

(2173, 100)

In [5]:
similar_up_origin = [i[0] for i in model.wv.most_similar("상승")]
similar_up_origin.append('상승')
similar_up_origin

['상승세', '하락', '급등', '오름세', '하락세', '보합', '약세', '폭', '강세', '둔화', '상승']

In [6]:
similar_up_expend = [(suo,model.wv.similarity('상승',suo)) for suo in similar_up_origin]
for suo in similar_up_origin:
    similar_suo = [i[0] for i in model.wv.most_similar(suo)]
    for ssuo in similar_suo:
        similar_up_expend.append((ssuo,model.wv.similarity('상승',ssuo)))

In [7]:
similar_up_expend = list(set(similar_up_expend))
similar_up_df = pd.DataFrame(similar_up_expend)
similar_up_df = similar_up_df.sort_values(by = 1, ascending = False,ignore_index = True)
similar_up_df.columns = ['keyword','similarity_with_상승']
similar_up_df

,keyword,similarity_with_상승
0,상승,1.000000
1,상승세,0.828450
2,하락,0.822300
3,급등,0.783167
4,오름세,0.774329
5,하락세,0.742564
6,보합,0.702734
7,약세,0.696168
8,폭,0.694132
9,강세,0.691680


In [8]:
similar_down_origin = [i[0] for i in model.wv.most_similar("하락")]
similar_down_origin.append('하락')
similar_down_origin

['하락세', '상승', '약세', '보합', '상승세', '오름세', '둔화', '급락', '급등', '안정세', '하락']

In [9]:
similar_down_expend = [(suo,model.wv.similarity('하락',suo)) for suo in similar_down_origin]
for suo in similar_down_origin:
    similar_suo = [i[0] for i in model.wv.most_similar(suo)]
    for ssuo in similar_suo:
        similar_down_expend.append((ssuo,model.wv.similarity('하락',ssuo)))

In [10]:
similar_down_expend = list(set(similar_down_expend))
similar_down_df = pd.DataFrame(similar_down_expend)
similar_down_df = similar_down_df.sort_values(by = 1, ascending = False,ignore_index = True)
similar_down_df.columns = ['keyword','similarity_with_하락']
similar_down_df

,keyword,similarity_with_하락
0,하락,1.000000
1,하락세,0.881505
2,상승,0.822300
3,약세,0.810064
4,보합,0.803016
5,상승세,0.792994
6,오름세,0.762696
7,둔화,0.748036
8,급락,0.719006
9,급등,0.686892


1. 중복된 키워드 지우기

In [11]:
up_word = []
_list = [i for i in similar_down_df['keyword']]
for i in similar_up_df['keyword']:
    if i not in _list:
        up_word.append(i)
    if len(up_word) == 5:
        break
up_word

['고공', '값']

In [12]:
down_word = []
_list = [i for i in similar_up_df['keyword']]
for i in similar_down_df['keyword']:
    if i not in _list:
        down_word.append(i)
    if len(down_word) == 5:
        break
down_word

['뚝', '회복', '추락', '쇼크', '최악']

2. 중복된 키워드가 up에 가까운지 down에 가까운지 확인하고 가까운 쪽으로

In [13]:
up_word = []
_list = [i for i in similar_down_df['keyword']]
for i in similar_up_df['keyword']:
    if i not in _list:
        up_word.append(i)
    else:
        if float(similar_up_df[similar_up_df['keyword'] == i]['similarity_with_상승']) >= float(similar_down_df[similar_down_df['keyword'] == i]['similarity_with_하락']):
            up_word.append(i)
    if len(up_word) == 5:
        break
up_word

['상승', '상승세', '급등', '오름세', '폭']

In [14]:
down_word = []
_list = [i for i in similar_up_df['keyword']]
for i in similar_down_df['keyword']:
    if i not in _list:
        down_word.append(i)
    else:
        if float(similar_down_df[similar_down_df['keyword'] == i]['similarity_with_하락']) >= float(similar_up_df[similar_up_df['keyword'] == i]['similarity_with_상승']):
            down_word.append(i)
    if len(down_word) == 5:
        break
down_word

['하락', '하락세', '약세', '보합', '둔화']